In [1]:
import os
from openai import *
import pandas as pd
import numpy as np
import pdb
import time
sys.path.append('../../../../infrastructure/tools')
from utilities import *
from plotting import *

In [2]:
raw_data = pd.read_csv("../../../data/complaints.csv")

In [3]:
raw_data = raw_data.fillna('unknown')

In [4]:
raw_data = raw_data.rename(columns={'Unnamed: 0':'complaint_id'})

In [5]:
raw_data.head()

,complaint_id,product,narrative
0,0,credit_card,purchase order day shipping amount receive pro...
1,1,credit_card,forwarded message date tue subject please inve...
2,2,retail_banking,forwarded message cc sent friday pdt subject f...
3,3,credit_reporting,payment history missing credit report speciali...
4,4,credit_reporting,payment history missing credit report made mis...


In [6]:
sample = raw_data.sample(frac=0.1, random_state=123).reset_index(drop=True)

In [7]:
with open("../../../../api_keys/openai_api.txt", "r") as password_file:  # Open in binary read mode
    
    password = password_file.read()


In [8]:
client = OpenAI(api_key=password)


In [9]:
sample['chatgpt_keyword_extraction'] = 'unknown'

In [10]:
labels = [p for p in sample['product'].unique()]

In [11]:
labels

['credit_reporting',
 'credit_card',
 'retail_banking',
 'debt_collection',
 'mortgages_and_loans']

In [12]:
sample.head(100)

,complaint_id,product,narrative,chatgpt_keyword_extraction
0,54641,credit_reporting,disputed error credit report multiple time sti...,unknown
1,55752,credit_reporting,filed dispute regard incorrect item credit rep...,unknown
2,39820,credit_reporting,decided get credit pulled due couple collectio...,unknown
3,76467,credit_reporting,sent paperwork containing undisputed overwhelm...,unknown
4,2686,credit_reporting,according fair credit reporting act section cr...,unknown
...,...,...,...,...
95,65869,credit_reporting,noticed credit report reflecting late payment ...,unknown
96,135883,debt_collection,account mine requested debt validated proof se...,unknown
97,141726,credit_reporting,may concern writing dispute fraudulent charge ...,unknown
98,110370,mortgages_and_loans,started refinance application submitted reques...,unknown


In [17]:
import time

t0 = time.time()

for i in range(1000):

    messages = [
    {"role": "system", "content": "You are a seasoned NLP data scientist and engineer. You will process text data as I request"}]

    request = 'Summarize the following context? context:' + f"{sample.at[i,'narrative']}"

    messages.append({"role": "user", "content": request})

    response = client.chat.completions.create(model="gpt-3.5-turbo", messages=messages)

    response_context = response.choices[0].message.content

    followup = f"Now tell me which label I should choose for the summary? Choose from the following keywords: {','.join(labels)}"

    messages.append({"role": "user", "content": followup})
        
    response = client.chat.completions.create(model="gpt-3.5-turbo", messages=messages)

    response_context = response.choices[0].message.content


    sample.at[i, 'chatgpt_keyword_extraction'] = response_context
    
t1 = time.time()

total = t1-t0

total

2662.8803248405457

In [14]:
# raw_data.chatgpt_keyword_extraction.value_counts()

In [15]:
# raw_data.save('../../../data/intermediate/raw_gpt_labeled.csv')